In [1]:
import sys
sys.path.insert(0, "../")
from pathlib import Path
import joblib
import lightning as pl

import pandas as pd
import numpy as np
from tqdm import tqdm

from artstract_ml.dataset import load_perceptual_dataset

pl.seed_everything(42)


/home/nicolas/.conda/envs/mosaic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


42

## Interpret Naive Bayes

In [27]:
train, valid, test = load_perceptual_dataset(Path("../exaplainability_samples/perception.json"))
clf_d = joblib.load("../output/nb/clf.joblib")
clf = clf_d["clf"]
fe = clf_d["fe"]
le = clf_d["le"]

In [34]:
from sklearn.metrics import classification_report

true = []
nb_pred = []
notempty_nb_pred = []

sample = test.iloc[1]
print("Predicting sample: ", sample.id)

X_sample = fe.transform(pd.DataFrame(sample[fe.feature_names_in_]).T).toarray()

prob = np.outer(clf.feature_log_prob_, X_sample)
features_with_value = np.array(["empty" not in x for x in fe.get_feature_names_out()])
feature_names = fe.get_feature_names_out()[features_with_value]
feature_log_probs = np.multiply(X_sample, clf.feature_log_prob_)[:, features_with_value]
class_probs = clf.class_log_prior_ + feature_log_probs.sum(axis=1)

idxs = feature_log_probs[class_probs.argmax()] < 0

Predicting sample:  danger_3


In [35]:
for idx, c in enumerate(le.classes_):
  print("Class: ", c)

  imp = zip(feature_names[idxs], np.exp(feature_log_probs[idx, idxs]))
  imp = sorted(imp, key=lambda x: x[1])
  for k, v in imp:
    print(k, v)
  print("-" * 10)
  

Class:  comfort
color_0_lightgray 0.0227028959599571
color_3_dimgray 0.047372184483375046
color_2_gray 0.07704683589560242
emotion_awe 0.09957096889524496
color_1_darkslategray 0.10672148730782986
art_style_Post-Impressionism 0.1673221308544869
action_sleeping 0.3010368251698249
----------
Class:  danger
color_0_lightgray 0.02552719200887903
color_3_dimgray 0.03218645948945617
art_style_Post-Impressionism 0.06659267480577136
color_2_gray 0.0899001109877914
color_1_darkslategray 0.10987791342952276
emotion_awe 0.1853496115427304
action_sleeping 0.3152053274139846
----------
Class:  death
color_0_lightgray 0.02233902759526939
color_3_dimgray 0.02233902759526939
art_style_Post-Impressionism 0.06219886114761282
color_1_darkslategray 0.08760402978537013
color_2_gray 0.13929040735873857
emotion_awe 0.18396846254927732
action_sleeping 0.3114323258869908
----------
Class:  fitness
color_0_lightgray 0.02291105121293801
color_3_dimgray 0.04447439353099731
art_style_Post-Impressionism 0.079514824